In [1]:
from extract_features import process_dataset, load_and_process_sample
from visualization import signal_viewer
from imu_pipeline import IMUPipeline
from pathlib import Path
import pandas as pd
import joblib
import os

In [2]:
signal_viewer(
    data_dir=Path('data/raw/train'),
    labels_csv=Path('data/train.csv')
)

In [3]:
# step 2
from extract_features import process_dataset
process_dataset('inference')

[INFO] No label CSV found for 'inference' – proceeding without labels.

Processing 'inference' dataset...


100%|██████████| 473/473 [00:08<00:00, 59.00it/s]


Processed 473 samples to data/inference.csv


,x_mean,x_std,x_max,x_min,x_range,x_skew,x_kurtosis,x_n_peaks,x_energy,y_mean,...,temperature,humidity,altitude,session_id,firmware_version,calibration_status,battery_level,gps_accuracy,network_type,device_model
0,2.578586,2.362892,8.999561,-5.200753,14.200314,0.193760,1.400440,31,1223.236383,2.995496,...,12,49,996,S603575,v1.3.0,ok,79,5.570705,4g,model_c
1,-0.114423,2.185548,10.289834,-5.283784,15.573618,2.164783,7.702143,5,478.971240,0.343485,...,22,75,607,S949312,v1.2.4,ok,81,3.181146,4g,model_c
2,0.061702,0.147340,0.291371,-0.208436,0.499806,-0.407912,-1.211002,15,2.551629,0.015451,...,30,52,482,S223545,v1.2.3,recent,95,2.987759,4g,model_b
3,1.354891,2.486652,11.114359,-6.365416,17.479775,0.764660,3.152818,15,801.916607,1.424555,...,11,36,463,S993755,v1.2.3,pending,47,9.744692,4g,model_a
4,0.061687,0.141320,0.278571,-0.246872,0.525443,-0.440889,-1.157643,17,2.377675,0.018567,...,13,41,759,S734746,v1.2.4,recent,37,3.622219,wifi,model_a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,0.060402,0.141580,0.300225,-0.230982,0.531207,-0.394775,-1.216041,14,2.369336,0.015395,...,15,57,750,S165342,v1.2.4,ok,54,9.761510,4g,model_c
469,0.159769,2.509925,8.347333,-9.574540,17.921873,0.287556,3.356591,10,632.524802,1.805974,...,12,76,900,S653588,v1.3.0,pending,23,8.203065,4g,model_a
470,-0.548699,2.591718,6.550702,-8.426525,14.977227,-0.191147,1.450593,11,701.807315,1.414847,...,22,77,11,S945979,v1.3.0,pending,80,5.033353,5g,model_b
471,-0.559437,1.851954,4.829766,-6.744202,11.573968,-0.103196,1.832355,9,374.270158,1.994104,...,29,50,831,S424057,v1.3.0,pending,83,1.703700,5g,model_c


In [4]:
# step 3
import pandas as pd
labels = pd.read_csv("data/manual_annotation/inference_labels.csv")
labels.shape

(473, 2)

In [5]:
# show sample
labels[:10]

,sample_id,label
0,b67dd849-bf1a-4083-a09b-e56eea5b58ca,collision
1,2fe95d3e-1db2-420e-a9b6-880a3e76251c,collision
2,3b0de448-dd4c-4300-949c-1d58b8f32795,collision
3,e38b956f-e34f-4527-8cd7-8890fd0b2b76,collision
4,284498b7-94bd-42fe-9e57-814554fab41f,collision
5,7965c2bd-e17a-4d7f-acfc-156805cc41a0,collision
6,0c898243-2688-47ed-babb-5001263ee6a3,collision
7,68115c10-b0e5-4cc6-88a6-d2db1d0c5502,collision
8,561523f2-8644-4785-89d6-16158dc7ffb8,collision
9,01b5f718-2978-468f-9789-89003ba755e6,collision


In [6]:
# step 4 - with modification
import joblib
from imu_pipeline import IMUPipeline

df_inf = pd.read_csv("data/inference.csv")
model = joblib.load("models/imu_pipeline.pkl")

# inference.csv and infrence labels arent sorted the same. Reindexing here to make it easier to visualise
df_inf = df_inf.set_index("sample_id").reindex(labels["sample_id"]).reset_index()
df_inf.shape

(473, 60)

In [7]:
preds = model.predict(df_inf)
probs = model.predict_proba(df_inf)
preds.shape, probs.shape

((473,), (473, 2))

In [8]:
# show pred sample
for i in zip(preds[:10],probs[:10]):
    print(i)

('collision', array([0.96, 0.04]))
('collision', array([0.97, 0.03]))
('collision', array([0.94, 0.06]))
('collision', array([1., 0.]))
('collision', array([1., 0.]))
('collision', array([0.98, 0.02]))
('collision', array([1., 0.]))
('collision', array([0.95, 0.05]))
('collision', array([1., 0.]))
('collision', array([1., 0.]))


In [9]:
# step 5
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [10]:
# metrics
accuracy = accuracy_score(labels["label"], preds)
precision = precision_score(labels["label"], preds, pos_label="collision")
recall = recall_score(labels["label"], preds, pos_label="collision")
f1 = f1_score(labels["label"], preds, pos_label="collision")

inf_metrics = {
    "Accuracy" : accuracy,
    "Precision" : precision,
    "Recall" : recall,
    "F1" : f1
}
inf_metrics

{'Accuracy': 0.6448202959830867,
 'Precision': 0.5885416666666666,
 'Recall': 0.9576271186440678,
 'F1': 0.7290322580645161}

In [11]:
# test set
df_test = pd.read_csv("data/test.csv")
test_labels = df_test.pop("label")
df_test.shape, test_labels.shape

((200, 60), (200,))

In [12]:
preds = model.predict(df_test)
probs = model.predict_proba(df_test)
preds.shape, probs.shape

((200,), (200, 2))

In [13]:
accuracy = accuracy_score(test_labels, preds)
precision = precision_score(test_labels, preds, pos_label="collision")
recall = recall_score(test_labels, preds, pos_label="collision")
f1 = f1_score(test_labels, preds, pos_label="collision")
test_metrics = {
    "Accuracy" : accuracy,
    "Precision" : precision,
    "Recall" : recall,
    "F1" : f1
}

In [14]:
metrics = {
    "inference_metrics" : inf_metrics,
    "test_metrics": test_metrics
}
metrics

{'inference_metrics': {'Accuracy': 0.6448202959830867,
  'Precision': 0.5885416666666666,
  'Recall': 0.9576271186440678,
  'F1': 0.7290322580645161},
 'test_metrics': {'Accuracy': 0.945,
  'Precision': 1.0,
  'Recall': 0.89,
  'F1': 0.9417989417989417}}

# ❓ Questions to Reflect On


- What do you observe when comparing the model’s predictions on the new data to its known performance?

- Is there anything in the data that might explain differences in behavior?

- Can you identify patterns or trends related to when the model succeeds or fails?

- Are there signals or features that seem to affect the model’s reliability?

- What could be done in the short term to handle the current situation?

- What are potential long-term steps to improve model performance in similar scenarios?

- What would you want to explore further if given more time or data?

- What assumptions did the model rely on during training — and are they still valid?